In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
import analyze_utils

save_dir =  '/home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input/'

r, all_losses = analyze_utils.load_results_and_cache_autoprompt_json(
    save_dir, include_losses=True, do_reranking=False, save_file='r.pkl')

getting dir_names...


100%|███████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 65.05it/s]


skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input/Jan_24_16_39_dyccffsqoslz/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input/Jan_24_17_11_hpxsekhobxhh/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input/Jan_24_17_16_kmttdoekvnsr/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input/R-interpretable-autoprompting.257628.out/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input/R-interpretable-autoprompting.257629.out/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__pref

In [4]:
len(r)

3749

In [5]:
pd.set_option('display.max_colwidth', None)

top_prompts = (
    r.groupby(['task_name', 'model_cls', 'seed'])
).first()
print(len(top_prompts))
top_prompts[['prefixes', 'reciprocal_rank', 'prefix_train_loss', 'prefix_train_acc']]

5


prefixes  \
task_name  model_cls  seed                                        
sst2_train autoprompt 2             1998""Customer echoes the "   
                      3      Traceparticularly Kardefineingerso   
           iprompt    1               "the film has the uncanny   
                      2          "an experience of a generation   
                      3              lee treats his audience as   

                            reciprocal_rank  prefix_train_loss  \
task_name  model_cls  seed                                       
sst2_train autoprompt 2        1.000000e-10           0.462155   
                      3        2.040816e-02           0.468878   
           iprompt    1        2.000000e-02           0.567237   
                      2        1.587302e-02           0.447807   
                      3        2.702703e-02           0.487298   

                            prefix_train_acc  
task_name  model_cls  seed                    
sst2_train autoprompt 2             0.453125  
                      3             0.468750  
           iprompt    1             0.484375  
                      2             0.468750  
                      3             0.484375

In [ ]:
top_prompts.groupby('model_cls').mean()['reciprocal_rank']

model_cls
autoprompt    0.668519
iprompt       0.888889
Name: reciprocal_rank, dtype: float64

In [ ]:
assert r['checkpoint'].unique()[0] == "EleutherAI/gpt-j-6B"

from iprompt import prompt_classification

model = prompt_classification.create_model(r['checkpoint'].unique()[0])

2023-01-23 11:53:02.077911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 11:53:02.284623: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-23 11:53:02.322536: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-23 11:53:03.500856: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [12]:
## Compute accuracy given correct prompt and save for each task.
import argparse
from tqdm.notebook import tqdm
from iprompt.data import get_data


data = []
print('calculating accs...')
n_shots = 1
batch_size = 16

"""
task_name: str = 'add_two',
 n_shots: int = 1,
 train_split_frac: float = None,
 max_dset_size: int = 10000,
 template_num_task_phrasing: int = 0,
 max_digit: int = 10,
 """

for _, output in tqdm(top_prompts.reset_index().iterrows(), total=len(top_prompts)):
    if 'ffb' not in output['task_name']: continue
    if output['model_cls'] == 'autoprompt': continue
    args = argparse.Namespace(**output)
    args.task_name = args.task_name.replace('_train', '_test')
    args.train_split_frac = 1.0 # take 100% of test set
    args.max_dset_size = 1000
    print(args.task_name)
    (dset, __dset_test), check_answer_func, descr = get_data(
        args.task_name, n_shots=n_shots, train_split_frac=args.train_split_frac,
        max_dset_size=args.max_dset_size, template_num_task_phrasing=0,
    )
    # if task_name == 'task107_splash_question_to_sql':
    #     batch_size = max(1, batch_size//4)
    ####   Manual prompt  ####
    descr = "" # tmp override
    manual_loss, manual_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=model, prefix=descr, multi_token=False, verbose=True,
        max_length=64, batch_size=64, tqdm_notebook=True,
        restrict_to_valid_answers=True,
    )
    print(output)
    print(f'\t{descr} || {manual_acc:.1f}%')
    ####   iPrompt prompt   ####
    iprompt_loss, iprompt_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=model, prefix=(output['prefixes'] + '. '), multi_token=False, verbose=True,
        max_length=64, batch_size=64, tqdm_notebook=True,
        restrict_to_valid_answers=True,
    )
    print(f'\t{output["prefixes"]} || {iprompt_acc:.1f}%')
    ####
    output['manual_acc'] = manual_acc
    output['iprompt_acc'] = iprompt_acc
    data.append(output)
    break

calculating accs...


  0%|          | 0/24 [00:00<?, ?it/s]

ffb_test
**loading data: financial_phrasebank // train


Found cached dataset financial_phrasebank (/home/jxm3/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
Loading cached shuffled indices for dataset at /home/jxm3/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141/cache-5e700914242e73b3.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1698 [00:00<?, ?ex/s]

  0%|          | 0/16 [00:00<?, ?it/s]

x_text[0]:'Input: "It estimates the operating profit to further improve from the third quarter ." Answer:'
y_text[0]:' positive\n'
Percent correct: 36.50
task_name                                                                                                                                                  ffb_train
model_cls                                                                                                                                                    iprompt
seed                                                                                                                                                               1
batch_size                                                                                                                                                        32
n_epochs                                                                                                                                                         100
                     

  0%|          | 0/16 [00:00<?, ?it/s]

x_text[0]:' \'\' Answer: neutral \'\': "The actions are expected of. Input: "It estimates the operating profit to further improve from the third quarter ." Answer:'
y_text[0]:' positive\n'
Percent correct: 65.90
	 '' Answer: neutral '': "The actions are expected of || 65.9%


## Loading with PromptSource

In [33]:
import promptsource
import promptsource.templates

imdb_prompts = promptsource.templates.DatasetTemplates('imdb')

pos_input = { "text": "\"What a wonderful film :) \"", "label": 1 }
neg_input = { "text": "\"This movie sucks!\"", "label": 0 }

for tn in imdb_prompts.all_template_names:
    print(tn)
    print('\t [+]', imdb_prompts[tn].apply(pos_input))
    print('\t [-]', imdb_prompts[tn].apply(neg_input))
    print()

Movie Expressed Sentiment
	 [+] ['"What a wonderful film :) " The sentiment expressed for the movie is', 'positive']
	 [-] ['"This movie sucks!" The sentiment expressed for the movie is', 'negative']

Movie Expressed Sentiment 2
	 [+] ['The following movie review expresses what sentiment? "What a wonderful film :) "', 'positive']
	 [-] ['The following movie review expresses what sentiment? "This movie sucks!"', 'negative']

Negation template for positive and negative
	 [+] ['"What a wonderful film :) " This is definitely not a', 'negative review.']
	 [-] ['"This movie sucks!" This is definitely not a', 'positive review.']

Reviewer Enjoyment
	 [+] ['"What a wonderful film :) " How does the reviewer feel about the movie?', 'They loved it']
	 [-] ['"This movie sucks!" How does the reviewer feel about the movie?', "They didn't like it!"]

Reviewer Enjoyment Yes No
	 [+] ['"What a wonderful film :) " Did the reviewer enjoy the movie?', 'Yes']
	 [-] ['"This movie sucks!" Did the reviewer en

In [29]:
imdb_prompts["Movie Expressed Sentiment"].apply({ "text": "This movie sucks!" })

['This movie sucks! The sentiment expressed for the movie is', '']